In [81]:
import sys,os
sys.path.append("..")

MONGO_URL="localhost:27017"
os.environ["MONGO_URL"] = MONGO_URL

MONGO_INITDB_ROOT_USERNAME="root" # 
os.environ["MONGO_INITDB_ROOT_USERNAME"] = MONGO_INITDB_ROOT_USERNAME

MONGO_INITDB_ROOT_PASSWORD="dr5[gnhn" # 
os.environ["MONGO_INITDB_ROOT_PASSWORD"] = MONGO_INITDB_ROOT_PASSWORD

DATABASE_NAME = "admin"
os.environ["MONGO_INITDB_DATABASE"] = DATABASE_NAME

MONGO_VENDORBOT_DATABASE = "vendorbot_db"
os.environ["MONGO_VENDORBOT_DATABASE"] = MONGO_VENDORBOT_DATABASE

MONGO_BOT_USERNAME="bot"
os.environ["MONGO_BOT_USERNAME"] = MONGO_BOT_USERNAME

MONGO_BOT_PASSWORD="botpsswd"
os.environ["MONGO_BOT_PASSWORD"] = MONGO_BOT_PASSWORD

from modules.db.dbconfig import db_client

from modules.db.dbschema import UserReagents

from modules.db.users import UsersCollection

users_collection = UsersCollection(db_client, MONGO_VENDORBOT_DATABASE)

user_id = 122267418
query = {"user_id": user_id}

print(db_client)
db_client.list_database_names()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', connecttimeoutms=5)


['admin', 'blacklist_rdkit_db', 'config', 'local', 'test_db', 'vendorbot_db']

In [82]:
users_collection.get_user(user_id)

{'_id': 122267418,
 'user_id': 122267418,
 'username': 'radical_subject',
 'firstname': 'Oleg',
 'lastname': 'Fedorov',
 'phone_number': '',
 'user_reagents': [{'reagent_internal_id': '3996fec2ca3540529c03d669d85836f8',
   'CAS': '3849-21-6',
   'SMILES': 'CCOC(=O)/C(C#N)=N/O',
   'sharing_status': 'shared',
   'timestamp': '19.09.2022 23:22'},
  {'reagent_internal_id': '26252ebc4b8b4f60915d0055ed2c11c2',
   'CAS': '153433-21-7',
   'SMILES': 'CCOC(=O)/C(C#N)=N/O[P+](N1CCCC1)(N1CCCC1)N1CCCC1.F[P-](F)(F)(F)(F)F',
   'sharing_status': 'shared',
   'timestamp': '19.09.2022 23:22'},
  {'reagent_internal_id': '29b1f3c734a144e288a5467ddc88c2ce',
   'CAS': '158014-03-0',
   'SMILES': 'CCOC(=O)/C(C#N)=N/[O-].[K+]',
   'sharing_status': 'shared',
   'timestamp': '19.09.2022 23:22'},
  {'reagent_internal_id': '3a1be71f458d4f8fa0633a4a11319619',
   'CAS': '958029-37-3',
   'SMILES': 'C[N+](C)=C(N1CCOCC1)n1n[n+]([O-])c2ncccc21.F[P-](F)(F)(F)(F)F',
   'sharing_status': 'shared',
   'timestamp': '19

In [83]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [84]:
# SMILES
smiles = 'CC[C@H]([NH3+])CO'



In [85]:
def neutralize_atoms(mol):
    '''
    atom neutralizer
    '''
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

from rdkit.Chem.rdmolops import GetFormalCharge
def charge_fixer(SMILES):
    '''
    Neutralize molecules atom by atom
    '''
    # Create RDKit molecular objects
    mol = Chem.MolFromSmiles(smiles)
    if GetFormalCharge(mol) != 0:
        neutralize_atoms(mol)
    return Chem.MolToSmiles(mol)

In [86]:
from rdkit import RDLogger, Chem
from rdkit.Chem import PandasTools, SanitizeMol

from mongordkit.Database import write
from mongordkit.Search import similarity, substructure
from mongordkit.Database import registration

In [87]:
rdmol = Chem.MolFromSmiles('Cc1ccccc1')
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id', ["a05785ae720b414ca41526baf22bc026", "a05785ae720b414ca41526baf22bc026"])
moldoc = scheme.generate_mol_doc(rdmol)

In [88]:
rdmol = Chem.MolFromSmiles('Cc1ccccc1')
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id', ["a05785ae720b414ca41526baf22bc026", "a05785ae720b414ca41526baf22bc026"])
moldoc = scheme.generate_mol_doc(rdmol)

print(moldoc)

{'rdmol': Binary(b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x07\x00\x00\x00\x80\x01\x06\x00`\x00\x00\x00\x01\x03\x06@(\x00\x00\x00\x03\x04\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x0b\x00\x01\x00\x01\x02h\x0c\x02\x03h\x0c\x03\x04h\x0c\x04\x05h\x0c\x05\x06h\x0c\x06\x01h\x0c\x14\x01\x06\x01\x06\x05\x04\x03\x02\x17\x04\x00\x00\x00\x00\x00\x00\x00\x16', 0), 'index': 'YXFVVABEGXRONW-UHFFFAOYSA-N', 'smiles': 'Cc1ccccc1', 'scheme': 'default', 'hashes': {'inchikey_standard': 'YXFVVABEGXRONW-UHFFFAOYSA-N', 'CanonicalSmiles': 'Cc1ccccc1'}, 'fingerprints': {}, 'value_data': {'reagent_internal_id': ['a05785ae720b414ca41526baf22bc026', 'a05785ae720b414ca41526baf22bc026']}}


In [89]:
result = write.WriteFromMolList(db_client[MONGO_VENDORBOT_DATABASE].users_collection, [rdmol], scheme=registration.MolDocScheme()) 

query = {'index': 'YXFVVABEGXRONW-UHFFFAOYSA-N'}
if result == 0:
    insertion_result = db_client[MONGO_VENDORBOT_DATABASE].users_collection.update_one(query, {"$set": moldoc}, upsert=True) 

print(insertion_result.acknowledged)

populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
True


In [90]:
print(response)

NameError: name 'response' is not defined

In [91]:
from modules.db.unique_molecules import UniqueMolecules

unique_molecules_collection = UniqueMolecules(db_client, MONGO_VENDORBOT_DATABASE)

In [92]:
rdmol = Chem.MolFromSmiles('Cc1ccccc1')
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id', ["a05785ae", "a05785ae72"])
moldoc = scheme.generate_mol_doc(rdmol)

print(moldoc)


{'rdmol': Binary(b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x07\x00\x00\x00\x80\x01\x06\x00`\x00\x00\x00\x01\x03\x06@(\x00\x00\x00\x03\x04\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x06@h\x00\x00\x00\x03\x03\x01\x0b\x00\x01\x00\x01\x02h\x0c\x02\x03h\x0c\x03\x04h\x0c\x04\x05h\x0c\x05\x06h\x0c\x06\x01h\x0c\x14\x01\x06\x01\x06\x05\x04\x03\x02\x17\x04\x00\x00\x00\x00\x00\x00\x00\x16', 0), 'index': 'YXFVVABEGXRONW-UHFFFAOYSA-N', 'smiles': 'Cc1ccccc1', 'scheme': 'default', 'hashes': {'inchikey_standard': 'YXFVVABEGXRONW-UHFFFAOYSA-N', 'CanonicalSmiles': 'Cc1ccccc1'}, 'fingerprints': {}, 'value_data': {'reagent_internal_id': ['a05785ae', 'a05785ae72']}}


In [93]:
unique_molecules_collection.update_molecule('YXFVVABEGXRONW-UHFFFAOYSA-N', moldoc)

In [94]:
SMILES = 'C[C@H](N)CO'
reagent_internal_id = 'a05785ae720b414ca41526baf22bc026'

def reagent_registration(SMILES, reagent_internal_id):

    molfile = Chem.MolFromSmiles(SMILES)
    scheme = registration.MolDocScheme()
    scheme.add_value_field('reagent_internal_id_list', [reagent_internal_id])
    moldoc = scheme.generate_mol_doc(molfile)

    result = write.WriteFromMolList(db_client[MONGO_VENDORBOT_DATABASE].unique_molecules_collection, [molfile], scheme=scheme) 

    query = {'index': moldoc['index']}
    # print(query)
    if result == 0:

        reagent_internal_id_list = unique_molecules_collection.get_molecule(moldoc['index'])["value_data"]['reagent_internal_id_list']
        
        if reagent_internal_id not in reagent_internal_id_list:
            reagent_internal_id_list.append(reagent_internal_id)

            scheme.add_value_field('reagent_internal_id_list', reagent_internal_id_list)

            moldoc = scheme.generate_mol_doc(molfile)
            unique_molecules_collection.update_molecule(moldoc['index'], moldoc)

    # print(insertion_result.acknowledged)
reagent_registration(SMILES, reagent_internal_id)

populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped


In [100]:
from rdkit import RDLogger, Chem
from mongordkit.Database import write
from mongordkit.Search import similarity, substructure

from rdkit.Chem import PandasTools, SanitizeMol
# Search.PrepareForSearch(rdkit_db, rdkit_db.molecules, rdkit_db.mfp_counts, rdkit_db.permutations)
substructure.AddPatternFingerprints(db_client[MONGO_VENDORBOT_DATABASE].unique_molecules_collection)
similarity.AddMorganFingerprints(db_client[MONGO_VENDORBOT_DATABASE].unique_molecules_collection, db_client[MONGO_VENDORBOT_DATABASE].mfp_counts)

# Generate 100 different permutations of length 2048 and save them in demo_db.permutations as separate documents.
similarity.AddRandPermutations(db_client[MONGO_VENDORBOT_DATABASE].permutations)

# Add locality-sensitive hash values to each documents in demo_db.molecules by splitting the 100 different permutations
# in demo_db.permutations into 25 different buckets.
similarity.AddLocalityHashes(db_client[MONGO_VENDORBOT_DATABASE].unique_molecules_collection, db_client[MONGO_VENDORBOT_DATABASE].permutations, 25)

# Create 25 different collections in db_demo each store a subset of hash values for molecules in demo_db.molecules.
similarity.AddHashCollections(db_client[MONGO_VENDORBOT_DATABASE], db_client[MONGO_VENDORBOT_DATABASE].unique_molecules_collection)


In [101]:
from operator import itemgetter
def similarity_search(database, smiles: str):
    """
    Ищем похожие реагенты умными функциями
    :param smiles:
    :return: отсортированный по похожести список реагентов
    """
    smiles = smiles.replace("|", "")  # вертикальная черта в SMILES - непонятно что несёт, и RDKIT ее не понимает, убираем ее
    mol = Chem.MolFromSmiles(smiles)

    if not mol:
        raise Exception("MolFromSmiles returned None")

    res = similarity.SimSearchAggregate(mol, database.unique_molecules_collection, database.mfp_counts, 0.1)

    if not res:
        return

    res = sorted(res, key=itemgetter(0), reverse=True)

    if res[0] is None:
        raise Exception(f"incorrect result: {str(res)}")

    return res

In [76]:
best_similarity_result_id = similarity_search(db_client[MONGO_VENDORBOT_DATABASE], SMILES)[0][1]

In [102]:
def get_most_similar_reagent(SMILES):

    best_similarity_result_id = similarity_search(db_client[MONGO_VENDORBOT_DATABASE], SMILES)[0][1]
    reagent_internal_id_list = unique_molecules_collection.get_molecule(best_similarity_result_id)["value_data"]['reagent_internal_id_list']

    return reagent_internal_id_list

In [106]:
SMILES = 'CCN=C=NCCCN(C)C'
get_most_similar_reagent(SMILES)

['8dd677dce68a4b65a164f4d6e882e038']

In [99]:
reagent_list = users_collection.get_reagents(122267418)

for reagent in reagent_list:
    SMILES = reagent['SMILES']
    reagent_internal_id = reagent['reagent_internal_id']
    reagent_registration(SMILES, reagent_internal_id)

populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 duplicates skipped
populating mongodb collection with compounds from list...
2 molecules successfully imported
0 du

In [3]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdmolfiles, AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
# for similarity search need mongo-rdkit
# from mongordkit.Search import similarity, substructure, utils
# from mongordkit import Search
# from mongordkit.Database import create, write


def update_rdkit_with_sialdrich (client, db_instance):
    # Disable rdkit warnings
    rdkit.RDLogger.DisableLog('rdApp.*')

    db_name = db_instance.DATABASE_NAME
    molecules_collection = client[db_name]["molecules"]


In [6]:
db_name = db_instance.DATABASE_NAME

NameError: name 'db_instance' is not defined

In [12]:
db_name

'timerbot_db'

In [5]:
client[db_name].molecules

NameError: name 'client' is not defined

In [19]:
client.drop_database("timerbot_db")